### THREADING in Python

* Creation
* Lock
* Async Write (Class Inherit)

In [ ]:
from threading import Thread
import time

def work(name,delay,repeat):
    print(name + ' Started')
    while repeat > 0:
        time.sleep(delay)
        print(name + ":" + str(time.ctime(time.time())))
        repeat -= 1
    print(name + " Completed")

#work('main thread',.1,10)  


t1 = Thread(target=work,args=("Worker1",.1,5))
t2 = Thread(target=work,args=("Worker2",.2,5))
t1.start()
#t1.join()
t2.start()
#t2.join()

print("Main completed")

In [ ]:
import threading
import time

tLock = threading.Lock()

def work(name, delay, repeat):
    print("Timer: " + name + " Started")
    
    while repeat > 0:
        #time.sleep(delay)
        #tLock.acquire()
        print(name + ": " + str(time.ctime(time.time())))
        #tLock.release()
        repeat -= 1


t1 = threading.Thread(target=work, args=("Timer1", .01, 20))
t2 = threading.Thread(target=work, args=("Timer2", .01, 20))
t2.start()
t1.start()

### THREADING - Games and Graphics
* Example using  **turtle**

In [1]:
import turtle
import threading 

def draw_spiral(my_turtle,line_len):
    if line_len > 0:
        my_turtle.forward(line_len)
        my_turtle.right(90)
        draw_spiral(my_turtle, line_len - 5)

def draw_playground(myTurtle):
    myTurtle.color('red', 'blue')
    myTurtle.begin_fill()
    while True:
        myTurtle.forward(170)
        myTurtle.left(200)
        if abs(myTurtle.pos()) < 1:
            break
    myTurtle.end_fill()

def StartDrawing():
    spiral = turtle.Turtle()
    playground = turtle.Turtle()
    spiral.pu()
    spiral.setx(-300)
    spiral.pd()

    #spiral_thread = threading.Thread(target=lambda: draw_spiral(spiral, 90))
    #playground_thread = threading.Thread(target=lambda: draw_playground(playground))
    #spiral_thread.start()
    #playground_thread.start()
    
    draw_spiral(spiral, 90)
    draw_playground(playground)

    turtle.Screen().exitonclick()

StartDrawing()

In [ ]:
import threading
import time

class AsyncWrite(threading.Thread):
    def __init__(self, text, out):
        threading.Thread.__init__(self)
        self.text = text
        self.out = out

    def run(self):
        f = open(self.out, "a")
        for i in range(100000):
            f.write(self.text + '\n')
        f.close()
        print("Finished Background file write to " + self.out)

if __name__ == '__main__':    
    
    start = time.time()
    backgrounds = []
    for i in range(10):
        message = 'Some text in file %d' % i 
        background = AsyncWrite(message, 'out%d.txt' % i)
        background.start()
        backgrounds.append(background)
    for i in range(10):
        backgrounds[i].join()
    
    print("Threading - Job time:", time.time() -start)
    '''
     
    start = time.time()
    for i in range(10):
        message = 'Some text in file %d' % i 
        out = 'sout%d.txt' % i
        f = open(out, "a")
        for i in range(100000):
            f.write(message + '\n')
        f.close()
        print("Finished file write to " + out)
    
    print("Seq - Job time:", time.time() -start)
    '''

### THREADING - WEB SCRAPING

* Example: Yahoo Finance
* http://finance.yahoo.com/q?s=GOOG

In [11]:
import requests
import re  
from threading import Thread
import time

YAHOO = 'http://finance.yahoo.com/q?s='

def getPrice(symbol):
    REGEX = re.compile('<span id="yfs_l84_' + symbol.lower() + '">(.+?)</span>')
    url = '%s%s' % (YAHOO, symbol)
    response = requests.get(url)
    html = str(response.content)  
    results = re.findall(REGEX,html)
    if results:
        print("The price of " + str(symbol) + " is " + str(results[0]))
    #print(html)

SYMBOLs = ['GOOG'] # ['AACAY','AAGIY','AAL','AALTF','AAMC','AAN','AAON']
#SYMBOLs = open(r'C:\NWU\MSiA 2016\PythonNotebook\PyProjects\Week09\symbols.txt').read().split()


print(SYMBOLs)

start = time.time()

for symbol in SYMBOLs:
    getPrice(symbol)
    
print("Job time:", time.time() -start)

'''
#--------------------------------
start = time.time()
threadlist = []

for i in range(100):
    symbol = SYMBOLs[i]
    t = Thread(target=getPrice,args=(symbol,))
    t.start()
    threadlist.append(t)

for t in threadlist:
    t.join()

print("Job time:", time.time() -start)
'''

['GOOG']


ConnectionError: HTTPConnectionPool(host='finance.yahoo.com', port=80): Max retries exceeded with url: /q?s=GOOG (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x0000000005FD72B0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

### THREADING - GUI User Experince

* Example: GUI library - tkinter


In [3]:
import tkinter
from threading import Thread
import time

myThread = None
stop_counting = False

def quit():
    global stop_counting
    stop_counting = True
    #global myThread #tkTop
    #myThread.stop()
    print("Counter Stopped ...")
    #global tkTop
    #tkTop.destroy()
    
def startCounter():
    global tkLabel
    global stop_counting
    print("Start Counting ...")
    for count in range(20):
        if not stop_counting:
            time.sleep(1)
            varLabel.set(str(count) + str(stop_counting)) 
            tkLabel.update_idletasks()

def runThread():
    global myThread
    global stop_counting
    stop_counting = False
    varLabel.set("runThread() called")
    myThread = Thread(target=startCounter)
    myThread.start()


tkTop = tkinter.Tk()
tkTop.geometry('300x200')
  
tkButtonQuit = tkinter.Button(
    tkTop,
    text="Quit",
    command=quit)
tkButtonQuit.pack()
 
tkButtonRunThread = tkinter.Button(
    tkTop,
    text="Start Counting",
    command=runThread) # startCounter) #
tkButtonRunThread.pack()
 
varLabel = tkinter.StringVar()
tkLabel = tkinter.Label(textvariable=varLabel)
tkLabel.pack()

tkinter.mainloop()

Start Counting ...
Counter Stopped ...
Start Counting ...
Counter Stopped ...


In [12]:
from bs4 import BeautifulSoup
import requests
import re

while True:
    symbol = input("Enter symbol: ")

    if symbol == 'stop':
        break

    url = 'http://finance.yahoo.com/q?s={}'.format(symbol)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        data = soup.find('span', attrs= {'id' : re.compile(r'yfs_.*?_{}'.format(symbol.lower()))})
        print('{} = {}'.format(symbol, data.text))
    except AttributeError:
        print("Unknown symbol: {}".format(symbol))

Enter symbol: GOOG


ConnectionError: HTTPConnectionPool(host='finance.yahoo.com', port=80): Max retries exceeded with url: /q?s=GOOG (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x0000000006337D30>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))